In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
import sys
import os
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.append(os.path.abspath('src'))
from utils import compare_metrics, plot_forecasts

import warnings
warnings.filterwarnings("ignore")

Country: IT; Frequency: M ; trans: light; method: 0; q: 99; algorithm: SW.

In [93]:
df = pd.read_excel("data/factor_data/ITdata_TR2.xlsx", parse_dates= True)

In [94]:
monthly_base_variables = [
    # Labor Market (Monthly)
    "UNETOT",   # Unemployment: Total
    "UNEO25",   # Unemployment: Over 25
    "UNEU25",   # Unemployment: Under 25

    # Financial Markets & Interest Rates
    "REER42",   # Real Exchange Rate
    "ERUS",     # Exchange Rate (US Dollar)
    "SHIX",     # Stock Price Index
    "IRT3M",    # 3-Months Interest Rates
    "IRT6M",    # 6-Months Interest Rates
    "LTIRT",    # Long-Term Interest Rates

    # Turnover
    "TRNMN",    # Turnover: Manufacturing
    "TRNCAG",   # Turnover: Capital Goods
    "TRNCOG",   # Turnover: Consumer Goods
    "TRNDCOG",  # Turnover: Durable Consumer Goods
    "TRNNDCOG", # Turnover: Non Durable Consumer Goods
    "TRNING",   # Turnover: Intermediate Goods
    "TRNNRG",   # Turnover: Energy
    "CAREG",    # Passenger Car Registrations

    # Prices (PPI & HICP)
    "PPICAG",   # PPI: Capital Goods
    "PPICOG",   # PPI: Consumer Goods
    "PPIDCOG",  # PPI: Durable Consumer Goods
    "PPINDCOG", # PPI: Non Durable Consumer Goods
    "PPING",    # PPI: Intermediate Goods
    "PPINRG",   # PPI: Energy
    "HICPOV",   # HICP: Overall Index
    "HICPNEF",  # HICP: No Energy & Food
    "HICPG",    # HICP: Goods
    "HICPIN",   # HICP: Industrial Goods
    "HICPSV",   # HICP: Services
    "HICPNG",   # HICP: Energy

    # Confidence Indicators
    "ICONFIX",  # Industrial Confidence
    "CCONFIX",  # Consumer Confidence
    "ESENTIX",  # Economic Sentiment
    "KCONFIX",  # Construction Sentiment
    "RTCONFIX", # Retail Confidence
    "SCONFIX",  # Services Confidence
    "BCI",      # Business Confidence Index
    "CCI",      # Consumer Confidence Index (OECD)

    # Monetary Aggregates
    "CURR",     # Currency
    "M1",       # Money Stock M1
    "M2"        # Money Stock M2
]

# dd the "_IT" suffix to match your dataset structure
monthly_vars_IT = [f"{var}_IT" for var in monthly_base_variables]

# We use intersection() to avoid errors if a specific column is missing in your CSV
columns_to_keep = ["Time"] +[col for col in monthly_vars_IT if col in df.columns]
df = df[columns_to_keep]

In [95]:
df = df.rename({"Time" : "observation_date"}, axis = 1)
filtered_df = df[df['observation_date'].between("2000-03-01", "2022-11-01")]

filtered_df

,observation_date,UNETOT_IT,UNEO25_IT,UNEU25_IT,REER42_IT,SHIX_IT,LTIRT_IT,TRNCAG_IT,TRNCOG_IT,TRNDCOG_IT,...,HICPSV_IT,HICPNG_IT,ICONFIX_IT,CCONFIX_IT,ESENTIX_IT,KCONFIX_IT,RTCONFIX_IT,SCONFIX_IT,BCI_IT,CCI_IT
2,2000-03-01,10.5,8.4,27.5,-1.090447,0.051356,5.58,-0.065468,-0.006609,-0.018631,...,-0.000015,0.016033,10.6,-5.4,119.0,-4.3,-10.9,26.9,102.6290,102.77900
3,2000-04-01,10.3,8.0,27.7,-0.611787,-0.086655,5.47,0.038492,0.020998,0.029430,...,-0.001415,-0.010972,10.2,-4.8,122.0,-0.9,10.5,30.5,102.6958,102.61820
4,2000-05-01,10.3,8.0,27.6,-0.620661,0.021817,5.67,-0.009156,-0.001300,-0.003228,...,0.000453,0.013431,11.5,-6.7,123.7,7.8,26.0,29.0,102.7392,102.46750
5,2000-06-01,10.2,8.0,27.5,0.911656,0.017620,5.51,-0.052608,-0.018373,-0.026202,...,0.002178,0.011615,10.4,-6.1,120.9,-2.1,-14.1,30.5,102.7115,102.47060
6,2000-07-01,10.0,7.8,27.2,0.396406,0.019114,5.59,0.084932,0.030009,0.026202,...,0.008858,0.021829,11.2,-5.4,116.6,1.5,-23.3,19.7,102.6477,102.55900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2022-07-01,8.0,7.0,23.4,-1.265788,-0.053747,3.36,0.025129,0.006926,-0.013435,...,0.004789,-0.002683,1.0,-27.9,100.3,2.2,11.4,4.6,101.2142,96.87597
271,2022-08-01,8.2,7.0,22.5,0.476865,0.048099,3.30,0.068380,0.038092,0.062903,...,0.003866,0.023335,-0.2,-24.6,99.9,-5.5,16.9,5.1,100.9613,96.87737
272,2022-09-01,8.0,7.0,23.9,-0.134489,-0.050603,4.14,-0.012090,-0.006667,-0.007650,...,0.003855,0.002868,-2.4,-26.4,95.6,3.2,14.8,-1.8,100.7470,97.01801
273,2022-10-01,7.8,6.9,22.8,2.785363,-0.010055,4.53,-0.007850,-0.018566,-0.071617,...,0.002407,0.179160,-2.9,-26.4,95.1,-2.2,13.9,0.0,100.6627,97.53803


In [96]:
df_target = pd.read_csv("data/processed/df_stationary.csv")
columns_to_keep = ["observation_date","LOG_Y_IPI", "DLOG_Y_IPI"]
df_target = df_target[columns_to_keep]
df_target

# Make both columns proper datetime
filtered_df["observation_date"] = pd.to_datetime(filtered_df["observation_date"])
df_target["observation_date"]   = pd.to_datetime(df_target["observation_date"])

# Now the merge will work
df = filtered_df.merge(df_target, on="observation_date")


In [ ]:
# suppose you have df and df_target as in the screenshot

# 1) merge on the common date column
df = df_target.merge(filtered_df, on="observation_date")  # inner join, dates already match

# 2) convert to datetime (if not already)
df["observation_date"] = pd.to_datetime(df["observation_date"])

# 3) move date to index
df = df.set_index("observation_date")

,LOG_Y_IPI,DLOG_Y_IPI,UNETOT_IT,UNEO25_IT,UNEU25_IT,REER42_IT,SHIX_IT,LTIRT_IT,TRNCAG_IT,TRNCOG_IT,...,HICPSV_IT,HICPNG_IT,ICONFIX_IT,CCONFIX_IT,ESENTIX_IT,KCONFIX_IT,RTCONFIX_IT,SCONFIX_IT,BCI_IT,CCI_IT
observation_date,,,,,,,,,,,,,,,,,,,,,
2000-03-01,4.842288,0.008340,10.5,8.4,27.5,-1.090447,0.051356,5.58,-0.065468,-0.006609,...,-0.000015,0.016033,10.6,-5.4,119.0,-4.3,-10.9,26.9,102.6290,102.77900
2000-04-01,4.849735,0.007447,10.3,8.0,27.7,-0.611787,-0.086655,5.47,0.038492,0.020998,...,-0.001415,-0.010972,10.2,-4.8,122.0,-0.9,10.5,30.5,102.6958,102.61820
2000-05-01,4.855490,0.005754,10.3,8.0,27.6,-0.620661,0.021817,5.67,-0.009156,-0.001300,...,0.000453,0.013431,11.5,-6.7,123.7,7.8,26.0,29.0,102.7392,102.46750
2000-06-01,4.853849,-0.001641,10.2,8.0,27.5,0.911656,0.017620,5.51,-0.052608,-0.018373,...,0.002178,0.011615,10.4,-6.1,120.9,-2.1,-14.1,30.5,102.7115,102.47060
2000-07-01,4.847260,-0.006589,10.0,7.8,27.2,0.396406,0.019114,5.59,0.084932,0.030009,...,0.008858,0.021829,11.2,-5.4,116.6,1.5,-23.3,19.7,102.6477,102.55900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,4.651626,0.005038,8.0,7.0,23.4,-1.265788,-0.053747,3.36,0.025129,0.006926,...,0.004789,-0.002683,1.0,-27.9,100.3,2.2,11.4,4.6,101.2142,96.87597
2022-08-01,4.675461,0.023835,8.2,7.0,22.5,0.476865,0.048099,3.30,0.068380,0.038092,...,0.003866,0.023335,-0.2,-24.6,99.9,-5.5,16.9,5.1,100.9613,96.87737
2022-09-01,4.651626,-0.023835,8.0,7.0,23.9,-0.134489,-0.050603,4.14,-0.012090,-0.006667,...,0.003855,0.002868,-2.4,-26.4,95.6,3.2,14.8,-1.8,100.7470,97.01801
